## Web Scarping using Selenium

This notebook contains the code to scrape data for 1000 popular books on www.gutenberg.com

Importing the required libraries

In [2]:
import pandas as pd
import re as re

Setting up selenium to browser and driver. I am using Firefox for the browser. The driver used can be download from [here](https://github.com/mozilla/geckodriver/releases)

In [3]:
from selenium import webdriver 
from selenium.webdriver.firefox.service import Service
s=Service("D:\Setups\geckodriver-v0.30.0-win64\geckodriver.exe")
driver = webdriver.Firefox(service=s)

Connecting to the website.

In [4]:
driver.get('https://www.gutenberg.org/ebooks/search/?sort_order=downloads')

Finding the required class to scrape for data regarding the books

In [ ]:
books=driver.find_elements_by_class_name('booklink')
len(books)
print(books[-1].text)


Extracting the author and total download information

In [ ]:
name=books[0].find_elements_by_class_name('title')[0].text
author=books[0].find_elements_by_class_name('subtitle')[0].text
#total_downloads=books[0].find_elements_by_class_name('extra')[0].text
print(name)
print(author)
#print(total_downloads)

Script to extract data from the page and iterate to next page.

In [ ]:
count = 0
index=0
df=pd.DataFrame(columns=['Name', 'Author', 'Total Downloads'])
name_list=[]
author_list=[]
total_downloads_list=[]
while True:
    if count==40:
        break
    count+=1
    print('page ',count)
    books = driver.find_elements_by_class_name('booklink')
    
    for book in books:
        try:
            name_list.append(book.find_elements_by_class_name('title')[0].text)
            try:
                author_list.append(book.find_elements_by_class_name('subtitle')[0].text)
               # df['Author'].append(author)
            except:
                author_list.append('Not availbale')
               # df['Author'].append(author)
            try:
                total_downloads_list.append(book.find_elements_by_class_name('extra')[0].text)
                #df['Total Downloads'].append(total_downloads)
            except:
                total_downloads_list.append('Not availbale')
               # df['Total Downloads'].append(total_downloads)
            
            
            index+=1
            
        except:
            pass
    try:    
        driver.find_elements_by_class_name('statusline')[0].find_elements_by_tag_name('a')[-1].click()
    except:
        break
    print('|'*100)
df["Name"]=name_list
df["Author"]=author_list
df["Total Downloads"]=total_downloads_list
print(df)


In [ ]:
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)
df['number']=df['Total Downloads'].apply(lambda x: find_number(x))
print("\Extracting numbers from dataframe columns:")
df.head()

In [ ]:
df.drop('Total Downloads', axis=1, inplace=True)


In [13]:
df = df.rename(columns={"number":"Total Downloads"})
df.head()

,Name,Author,Total Downloads
0,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,78200
1,Pride and Prejudice,Jane Austen,55091
2,The Scarlet Letter,Nathaniel Hawthorne,37178
3,Alice's Adventures in Wonderland,Lewis Carroll,28310
4,Dracula,Bram Stoker,25993


Saving the dataframe to csv file

In [27]:
df.to_csv('popular_books.csv')